Notebook to do all the plotting in

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

per dataset - subset - n

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

testing_results = pd.read_csv('testing_results.csv')
testing_results = testing_results.fillna(0)

# narrow down dataset
# testing_results = testing_results[testing_results['dataset'] == 'IID']

testing_results = testing_results[
                                #     (testing_results['dataset'] == 'causalAssembly')
                                #   & (testing_results['subset'] == 'custom_line')
                                  (testing_results['date'] >= '2024-06-25 16:00:00')
                                  ]

# clean up testing data
testing_results.drop_duplicates(subset=['method', 'n'], inplace=True)
testing_results.sort_values(by='n', inplace=True)

# specify metrics
metrics = ['shd', 'recall', 'precision', 'runtime']  

for dataset in testing_results['dataset'].unique():
    dataset_results = testing_results[testing_results['dataset'] == dataset]
    
    for subset in dataset_results['subset'].unique():
        subset_results = dataset_results[dataset_results['subset'] == subset]
        
        for n in subset_results['n'].unique():
            subset_results_n = subset_results[subset_results['n'] == n]
            
            # Create a figure with the desired number of subplots (one for each metric)
            fig, axes = plt.subplots(1, len(metrics), figsize=(20, 4))

            # Set the overall title
            fig.suptitle(f'{dataset} - {subset} | n = {n}', fontsize=16)
            
            for i, metric in enumerate(metrics):
                ax = axes[i]
                
                for method in subset_results_n['method'].unique():
                    method_results = subset_results_n[subset_results_n['method'] == method]
                    ax.scatter(method_results[metric], method_results['method'], label=method)
                
                # ax.set_title(f'{dataset} - {subset} | n = {n} | {metric}')
                ax.set_xlabel(metric)
                if metric not in ('shd', 'runtime'):
                    ax.set_xticks(np.arange(0, 1.1, 0.1))
                ax.set_ylabel('method')
                ax.legend()
            
            plt.tight_layout()
            plt.show()

per dataset - subset: over all metrics
comparison of all adj_matrix vs ground_truth

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Specify algorithms
algorithms = ['NotearsLinear', 'DagmaLinear', 'GAE', 'DirectLiNGAM', 'PC']

# Specify sample sizes
sample_sizes = [500]

# Specify dataset
dataset = 'IID'
subset = 'IID3'

for sample_size in sample_sizes:

    # Create figure with as many subplots as algorithms plus one for the ground truth
    fig, axes = plt.subplots(1, len(algorithms) + 1, figsize=(20, 4), gridspec_kw={'width_ratios': [1] * len(algorithms) + [1]})

    # Set the overall title
    fig.suptitle(f'{dataset} - {subset} - sample size: {sample_size}', fontsize=20)

    # Create a subplot for each algorithm
    for i, algorithm in enumerate(algorithms):
        ax = axes[i]
        
        # Get the data
        adj_matrix = pd.read_csv(f'plots/adj_matrix_csv/{dataset}/{subset}/{algorithm}_{sample_size}.csv').to_numpy()

        ax.set_title(algorithm, y= 1.05, fontsize = 18)
        map1 = ax.imshow(adj_matrix, cmap='Greys', interpolation='none')

    # Also plot the ground truth
    ax_ground_truth = axes[-1]
    adj_matrix_ground_truth = pd.read_csv(f'data/{dataset}/{subset}/{subset}_ground_truth.csv').to_numpy()
    
    ax_ground_truth.set_title('Ground Truth', y=1.05, fontsize = 18)
    map2 = ax_ground_truth.imshow(adj_matrix_ground_truth, cmap='Greys', interpolation='none')

    # Add colorbar
    cbar = fig.colorbar(map2, ax=axes, location='right', aspect=40, fraction=0.02, pad=0.04)
    cbar.ax.set_position([ax_ground_truth.get_position().x1 + 0.01, ax_ground_truth.get_position().y0, 0.02, ax_ground_truth.get_position().height])
    
plt.show()


comparison of runtimes

In [ ]:
import matplotlib.pyplot as plt

algorithms = ['NotearsLinear', 'DagmaLinear', 'GAE', 'DirectLiNGAM', 'PC']

sample_sizes = [500, 1000, 2000, 5000, 10000]

datasets = ['IID', 'causalAssembly']
subsets = ['IID3', 'custom_line']

# load data
testing_results = pd.read_csv('testing_results.csv')
testing_results = testing_results.fillna(0)

# narrow down dataset
testing_results = testing_results[
                                    (testing_results['dataset'].isin(datasets))
                                  & (testing_results['subset'].isin(subsets))
                                #   (testing_results['date'] >= '2024-06-25 16:00:00')
                                  ]

fig, axs = plt.subplots(1, len(subsets), figsize = (20,4))
fig.suptitle(f'Vergleich von Laufzeiten in s von den Datensets {subsets[0]} und {subsets[1]} ', fontsize=20, y= 1.05)

for dataset in testing_results['dataset'].unique():
    for subset in testing_results[testing_results['dataset'] == dataset]['subset'].unique():
        subset_results = testing_results[(testing_results['dataset'] == dataset) & (testing_results['subset'] == subset)]

        # print (subset_results)

        # plot the runtime over n

        # Create a dictionary to store the data for each method
        method_data = {}
        
        for method in subset_results['method'].unique():
          method_results = subset_results[subset_results['method'] == method]
          x = method_results['n']
          y = method_results['runtime']
          
          # Store the data for each method
          method_data[method] = (x, y)
        
        # Plot the data for each method
        for method, (x, y) in method_data.items():
          ax.plot(x, y, label=method)
          ax.scatter(x, y)

          ax = axs[subsets.index(subset)]
          ax.set_title(subset, fontsize=18)
          ax.set_xlabel('n')
          ax.set_ylabel('runtime (s)')
          ax.set_xticks(sample_sizes)
          ax.tick_params(axis='x', labelrotation=90)
          ax.legend()

